[Inspiration](http://lenkiefer.com/2018/06/24/plotting-house-price-and-income-trends/)

In [3]:
import pybea
import pandas as pd
from config import api_key

Pull state names/abbreviations from Wikipedia; clean data and export it as .csv file for use in MySQL database

In [4]:
states = pd.read_html('https://en.wikipedia.org/wiki/List_of_U.S._state_abbreviations')[0]
states = states[12:]
states.drop(labels = [1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], axis = 1, inplace = True)
states = states.rename(index=str, columns={0 : "Name", 3 : "Abbreviation"})
states.reset_index(drop = True, inplace = True)
states.to_csv('States.csv')

Pull U.S. regional income data for 2018 using BEA API

In [5]:
regional_income_2018 = pybea.get_data(api_key,
                       DataSetName='RegionalIncome',
                       TableName='SQ1',
                       LineCode=3,
                       GeoFips='STATE',
                       Year=['2018'],
                       ResultFormat='JSON')

Strip asterisks from Alaska and Hawaii 'GeoNames'

In [6]:
regional_income_2018['GeoName'] = regional_income_2018['GeoName'].str.strip(' *')

Drop unwanted columns

In [7]:
regional_income_2018.drop(labels=['CL_UNIT',
                                  'Code',
                                  'GeoFips',
                                  'NoteRef',
                                  'UNIT_MULT'],
                          axis = 1,
                          inplace = True)

Export cleaned regional income data as .csv file for use in MySQL

In [8]:
regional_income_2018.to_csv('Regional Income.csv')

Import housing price index data, which was downloaded as an .xls file and saved as a .csv file

In [9]:
housing_price_index = pd.read_csv('Housing Price Index (Original).csv')

Drop unwanted columns

In [10]:
housing_price_index.drop(labels=['index_nsa',
                                 'Warning'],
                         axis = 1,
                         inplace = True)

Combine 'yr' and 'qtr' in the desired format (e.g., '2018Q1') for insertion into created 'TimePeriod' column

In [11]:
def quarterizer(row):
    year = row['yr']
    quarter = row['qtr']
    str_rep = str(year) + 'Q' + str(quarter)
    return str_rep

housing_price_index['TimePeriod'] = housing_price_index.apply(quarterizer, axis = 1)

Drop unwanted columns

In [12]:
housing_price_index.drop(labels=['yr',
                                 'qtr'],
                         axis = 1,
                         inplace = True)

Export transformed housing price index data as a .csv file for use in MySQL

In [13]:
housing_price_index.to_csv('Housing Price Index (Transformed).csv')

Import tables joined in MySQL, clean data, calculate 'affordability index' for insertion into appropriately-named and newly-created column, and export tranformed data as a .csv file for later use

In [15]:
merged_data = pd.read_csv('Joined Output (MySQL Export).csv')

merged_data = merged_data.rename(index=str, columns={'index_sa' : "Housing Price Index",
                                                     'DataValue' : "Regional Income",
                                                     'GeoName' : 'State',
                                                     'TimePeriod' : 'Yr. & Qtr.'})

merged_data['Affordability Ratio'] = (merged_data['Housing Price Index'] / merged_data['Regional Income']) * 100

merged_data.to_csv('Joined Output (Transformed).csv')